In [1]:
import sys
sys.path.append('./venv/lib/python3.6/site-packages/')

import yaml
import weibopy as wb


with open('./credentials.yml') as f:
    cred = yaml.load(f)

oauth = wb.OAuthHandler(**cred['weibo']).authorize()
api = wb.API(oauth=oauth)
data, status = api.statuses.user_timeline()

Please load the link https://api.weibo.com/oauth2/authorize?response_type=code&client_id=3677147061&redirect_uri=https%3A%2F%2Fwww.greenpeace.org%2F&state=51NST3m8efJz4aeOjuxOOfQcTUbiFs to your browser and authorize weibo API access.
Enter the full callback URL: https://www.greenpeace.org/usa/?state=51NST3m8efJz4aeOjuxOOfQcTUbiFs&code=532cc267f5aa1b2f5972c9d009267a80
UID: 1292378751
Access Token expires at: 2023/07/07 23:06:39


In [2]:
from utils import MongoWriter
writer = MongoWriter(**cred['mongo'])

In [40]:
from utils import MongoWriter


COLLECTION_MAPPING = {
    'Status': 'statuses',
    'Comment': 'comments',
    'User': 'users'
}


class WeiboData(object):

    _data = dict()
    _keys = ()

    def __init__(self, data: dict):
        print('test', data['created_at'])
        self._data['_id'] = data.get('id', 0)
        for key in self._keys:
            if key in self._data:
                continue
            print(key, data[key])
            if key in data:
                self._data[key] = data[key]
        print(self._data)

    def get(self, key, *args, **kwargs):
        return self._data.get(key, *args, **kwargs)

    def get_data(self):
        return self._data

    def write(self, writer: MongoWriter):
        collection = COLLECTION_MAPPING[self.__class__.__name__]
        result = writer.write(self._data, collection=collection)
        if result == -1:
            logging.warning('Duplicate id (_id:{}) found in collection.'.format(self._data['_id']))
        else:
            logging.info('Data (_id:{}) inserted into {}'.format(result, collection))


class Status(WeiboData):

    _keys = 'created_at', 'text'
#     , 'textLength', 'source', 'favorited', 'truncated', 'pic_urls', 'geo', 'is_paid', \
#            'annotations', 'reposts_count', 'comments_count', 'attitudes_count', 'isLongText', 'multi_attitude', \
#            'hide_flag', 'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', \
#            'hot_weibo_tags', 'text_tag_tips', 'userType', 'positive_recom_flag', 'content_auth', 'gif_ids', \
#            'is_show_bulletin', 'comment_manage_info'

    def __init__(self, data: dict, write_obj: bool=False, writer: MongoWriter=None):
        self._data = dict()
        if 'user' in data:
            user = User(data['user'])
            self._data['uid'] = user.get('_id', 0)
            if write_obj and writer:
                user.write(writer=writer)
        elif 'uid' in data:
            self._data['uid'] = data.get('uid', 0)
        if 'retweeted_status' in data:
            retweeted_status = Status(data=data['retweeted_status'])
            self._data['retweeted_id'] = retweeted_status.get('_id')
            if write_obj and writer:
                retweeted_status.write(writer=writer)
        super().__init__(data=data)

In [41]:
for s_data in data['statuses']:
    status = Status(s_data)
#     print(status.get_data())
#     status.write(writer=writer) 

test Wed Jun 13 12:27:48 +0800 2018
created_at Wed Jun 13 12:27:48 +0800 2018
text 南极是地球上最后一片净土。然而绿色和平在南极海洋中却发现了微塑料和有毒有害氟化物。南极海洋以及生活在那里的野生动物需要更好的保护，设立海洋保护区是#守护南极# 的第一步。此次，@李光洁 与绿色和平携手完成了一次特别的艺术创作，戳视频，听他娓娓道来《企鹅人漂流记》幕后的故事。 ...全文： http://m.weibo.cn/1292378751/4250426327276250 ​
{'uid': 1292378751, '_id': 4250426327276250, 'created_at': 'Wed Jun 13 12:27:48 +0800 2018', 'text': '南极是地球上最后一片净土。然而绿色和平在南极海洋中却发现了微塑料和有毒有害氟化物。南极海洋以及生活在那里的野生动物需要更好的保护，设立海洋保护区是#守护南极# 的第一步。此次，@李光洁 与绿色和平携手完成了一次特别的艺术创作，戳视频，听他娓娓道来《企鹅人漂流记》幕后的故事。 ...全文： http://m.weibo.cn/1292378751/4250426327276250 \u200b'}
test Fri Jun 08 22:33:09 +0800 2018
created_at Fri Jun 08 22:33:09 +0800 2018
text 近期，绿色和平在从南极采集的水样、雪样和海面拖网样品中发现并证实了——南极正在被微塑料和有毒有害化学品PFASs（全氟和多氟烷基化合物）污染！地球是人类的家园，不是塑料的家园，是时候和我们一起行动起来，对抗这个来势凶猛的敌人。#摆脱塑缚# #减塑生活#  http://t.cn/RB7GgKO ​
{'uid': 1292378751, '_id': 4248766729509158, 'created_at': 'Fri Jun 08 22:33:09 +0800 2018', 'text': '近期，绿色和平在从南极采集的水样、雪样和海面拖网样品中发现并证实了——南极正在被微塑料和有毒有害化学品PFASs（全氟和多氟烷基化合物）污染！地球是人类的家园，不是塑料的家园

In [6]:
# data, status = api.statuses.user_timeline(max_id=4248766729509158)

for s_data in data['statuses']:
    print(s_data['created_at'])
    status = Status(s_data)
    print(status._data['_id'], status._data['created_at'])

Wed Jun 13 12:27:48 +0800 2018
4250426327276250 Fri Aug 28 16:34:18 +0800 2009
Fri Jun 08 22:33:09 +0800 2018
4248766729509158 Fri Aug 28 16:34:18 +0800 2009
Tue Jun 05 16:03:13 +0800 2018
4247581435297686 Fri Aug 28 16:34:18 +0800 2009
Fri Jun 01 15:05:29 +0800 2018
4246117354311557 Fri Aug 28 16:34:18 +0800 2009
Fri May 25 17:30:03 +0800 2018
4243617021947926 Fri Aug 28 16:34:18 +0800 2009
Sat May 19 10:56:29 +0800 2018
4241343654479917 Fri Aug 28 16:34:18 +0800 2009


In [54]:
# data, status = api.comments.show(id=4250426327276250)

for s_data in data['comments']:
    print(s_data['id'], s_data['floor_number'], s_data['created_at'], s_data['status']['id'], s_data.get('reply_comment', {}).get('id'))
#     s_data['text']

4259206032691689 60 Sat Jul 07 17:55:13 +0800 2018 4250426327276250 None
4257895908341169 0 Wed Jul 04 03:09:15 +0800 2018 4250426327276250 4250620259670924
4257838387854399 59 Tue Jul 03 23:20:41 +0800 2018 4250426327276250 None
4254975343009732 58 Tue Jun 26 01:43:58 +0800 2018 4250426327276250 None
4254198877519865 57 Sat Jun 23 22:18:34 +0800 2018 4250426327276250 None
4252939264331013 56 Wed Jun 20 10:53:18 +0800 2018 4250426327276250 None
4251844676740213 55 Sun Jun 17 10:23:49 +0800 2018 4250426327276250 None
4251818151961743 54 Sun Jun 17 08:38:24 +0800 2018 4250426327276250 None
4251806529390153 53 Sun Jun 17 07:52:15 +0800 2018 4250426327276250 None
4251596248239347 51 Sat Jun 16 17:56:38 +0800 2018 4250426327276250 None
4251449976667290 50 Sat Jun 16 08:15:25 +0800 2018 4250426327276250 None
4251428078415332 49 Sat Jun 16 06:48:24 +0800 2018 4250426327276250 None
4251288319498696 48 Fri Jun 15 21:33:03 +0800 2018 4250426327276250 None
4251261500654057 47 Fri Jun 15 19:46:29 

In [47]:
s_data.keys()

dict_keys(['created_at', 'id', 'rootid', 'floor_number', 'text', 'disable_reply', 'user', 'mid', 'idstr', 'status'])

In [ ]:
from weibo_data import *
from utils import MongoWriter


def main(cred):
    writer = MongoWriter(**cred['mongo'])
    oauth = wb.OAuthHandler(**cred['weibo']).authorize()
    api = wb.API(oauth=oauth)
    data, _ = api.statuses.user_timeline()
    for s_data in data['statuses']:
        status = Status(s_data)
        print(status)
        status.write(writer=writer)


if __name__ == '__main__':
    with open('./credentials.yml') as f:
        cred = yaml.load(f)
    main(cred=cred)

In [ ]:
api = wb.API(oauth)
data, status = api.statuses.user_timeline(count=100)

In [ ]:
statuses = data.get('statuses', [])

In [ ]:
for status in statuses:
    print(status.get('created_at'), status.get('id'))

In [ ]:
data, status = api.statuses.user_timeline(count=100, max_id=4248766729509158, trim_user=1)

In [ ]:
statuses2 = data.get('statuses', [])

In [ ]:
for status in statuses2:
    print(status.get('created_at'), status.get('id'))

In [ ]:
status = statuses2[2]
status

In [ ]:
status.keys()

In [ ]:
data, _ = api.statuses.show(id='4243617021947926')

In [ ]:
data.keys()

In [ ]:
data['user']

In [ ]:
data, _ = api.users.show(uid='1292378751')

In [ ]:
data

In [ ]:
help(api.users.show)

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url', 
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count', 
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled',
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel', 'verified_contact_name', 
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', 'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'mbtype', 'mbrank', 'block_word', 'block_app', 'credit_score', 'user_ability', 'urank', 'story_read_state', 'vclub_member'

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url',
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count',
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled', 
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel','verified_contact_name',
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', \
            'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'block_word', 'block_app', 'credit_score',\
            'user_ability', 'urank', 'story_read_state',

In [ ]:
data, _ = api.comments.show(id='4250426327276250')

In [ ]:
comments = data['comments']
len(comments)

In [ ]:
for c in comments:
    print(c['created_at'], c['id'])

In [ ]:
data2, _ = api.comments.show(id='4250426327276250', max_id='4250427878887380')

In [ ]:
comments = data2['comments']

for c in comments:
    print(c['created_at'], c['id'])